In [1]:
%pip install --quiet "evadb[vision,notebook]"
import evadb
cursor = evadb.connect().cursor()
"""
Issues Encountered:
cant have negative numbers in multidimensional arrays
cant have camel case words
cant have float attributes
"""

Note: you may need to restart the kernel to use updated packages.


'\nIssues Encountered:\ncant have negative numbers in multidimensional arrays\ncant have camel case words\ncant have float attributes\n'

In [2]:
### Set up Yolo UDF
cursor.query("""
            CREATE UDF IF NOT EXISTS Yolo
            TYPE  ultralytics
            'model' 'yolov8m.pt';
    """).df() 

,0
0,"UDF Yolo already exists, nothing added."


In [3]:
### Set up Monodepth UDF
cursor.query(""" 
            CREATE UDF IF NOT EXISTS MonodepthDetection
            IMPL'/home/youse/apperception/eva/udfs/monodepth_detection.py';
    """).df()

,0
0,"UDF MonodepthDetection already exists, nothing..."


In [4]:
### Load video
video_name = "boston-seaport-scene-0757-CAM_FRONT.mp4"
scene = "scene-0757"
video_path = "/data/processed/full-dataset/mini/videos/"
cursor.query("DROP TABLE IF EXISTS ObjectDetectionVideos;").df()
cursor.load(file_regex=video_path + video_name, format="VIDEO", table_name='ObjectDetectionVideos').df()

,0
0,Number of loaded VIDEO: 1


In [5]:
cursor.query("DROP TABLE IF EXISTS CameraConfigs;").df()
cursor.create_table("CameraConfigs", if_not_exists=True, columns="""
                cameraid TEXT(15),
                framenum INTEGER,
                cameratranslation NDARRAY FLOAT32(ANYDIM),
                camerarotation TEXT(100),
                cameraintrinsic NDARRAY FLOAT32(ANYDIM),
                egoheading INTEGER
            """).df()


""


In [6]:
### Add Camera Configs
%cd ..
from apperception.database import database
import pandas as pd


result = database.execute(f"SELECT cameraId, frameNum, cameraTranslation, cameraRotation, cameraIntrinsic, egoHeading FROM Cameras WHERE cameraId = '{scene}'")
df = pd.DataFrame()
for r in result:
    cameraId, frameNum, cameraTranslation, cameraRotation, cameraIntrinsic, egoHeading = r
    cameraTranslation = list(cameraTranslation)
    cursor.query(f"""INSERT INTO CameraConfigs (cameraid, framenum, cameratranslation, camerarotation, cameraintrinsic, egoheading) VALUES
                                ('{cameraId}', {frameNum}, {cameraTranslation}, '{cameraRotation}', {cameraIntrinsic}, {egoHeading});""").df()
df
    

/home/youse/apperception


""


In [7]:
cursor.query("SELECT id, egoheading FROM ObjectDetectionVideos JOIN CameraConfigs ON id = framenum").df()

,objectdetectionvideos.id,cameraconfigs.egoheading
0,1,63.418377
1,2,63.410492
2,3,63.406160
3,4,63.397694
4,5,63.397503
5,6,63.394688
6,7,63.392323
7,8,63.392437
8,9,63.391420


In [6]:
### Run query
cursor.query("""
            SELECT id, Yolo(data) FROM ObjectDetectionVideos
            WHERE Intersection(Location(Yolo(data)., MonodepthDetection(data)))
""").df()


UnexpectedToken: Unexpected token Token('COMMA', ',') at line 3, column 52.
Expected one of: 
	* ID
	* REVERSE_QUOTE_ID
Previous tokens: [Token('DOT', '.')]
